# Quickstart

In the quickstart guide, we demonstrate how to create a basic data labeling agent.

Let's use a dataset of product reviews stored in pandas dataframe, and trying to understand subjectiveness of given texts:

In [24]:
import pandas as pd
texts = [
    "The mic is great.",
    "Will order from them again!",
    "Not loud enough and doesn't turn on like it should.",
    "The phone doesn't seem to accept anything except CBR mp3s",
    "All three broke within two months of use."
]
df = pd.DataFrame(texts, columns=['text'])
df

,text
0,The mic is great.
1,Will order from them again!
2,Not loud enough and doesn't turn on like it sh...
3,The phone doesn't seem to accept anything exce...
4,All three broke within two months of use.


Since the goal is to classify whether each review represents objective statement or subjective opionion, let's define our agent who should classify given dataset into the categories. The agent labeling abilities are defined as **"Skills"**, and each agent can possess many different skills. 

For simplicity, let's create an agent with only one labeling skill:

In [25]:
from adala.agents import SingleShotAgent
from adala.datasets import DataFrameDataset
from adala.skills import LabelingSkill

agent = SingleShotAgent(
    # connect to a dataset
    dataset=DataFrameDataset(df=df),
    
    # define the agent's labeling skill
    skill=LabelingSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels=['Subjective', 'Objective']
    )
)

agent.greet()


Hi, I am a single shot agent.
I can use a single skill to process a dataset.
For example, you can instruct me to label your dataset.
I have a skill name='subjectivity_detection' description='Understanding subjective and objective statements from text.' prompt_template="    {{>instructions}}\n    \n    Input: {{text}}\n    Output: {{select 'predictions' options=labels logprobs='score'}}\n    " instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.' prediction_field='predictions' labels=['Subjective', 'Objective'] which can label your dataset.


Now let's run our agent on a given dataset

In [26]:
run_results = agent.run()
pd.concat((df, run_results.experience.predictions), axis=1)

100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 49.62it/s]


,text,ground_truth,predictions,score
0,The mic is great.,NaN,Subjective,"{'Subjective': -0.04570976299999997, 'Objectiv..."
1,Will order from them again!,NaN,Subjective,"{'Subjective': -0.31326162999999996, 'Objectiv..."
2,Not loud enough and doesn't turn on like it sh...,NaN,Subjective,"{'Subjective': -0.10020652400000002, 'Objectiv..."
3,The phone doesn't seem to accept anything exce...,NaN,Subjective,"{'Subjective': -0.674278, 'Objective': -0.7123..."
4,All three broke within two months of use.,NaN,Objective,"{'Subjective': -3.1679988, 'Objective': -0.042..."


We may observe some errors in the predicted dataset. Let's fix them and create _ground truth_ signal:

In [27]:
df.loc[0, 'ground_truth'] = 'Subjective'
df.loc[1, 'ground_truth'] = 'Subjective'
df.loc[2, 'ground_truth'] = 'Objective'
df.loc[3, 'ground_truth'] = 'Objective'
df.loc[4, 'ground_truth'] = 'Objective'

and then we can ask our agent to learn and improve itself:

In [28]:
learn_results = agent.learn()

100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 54.83it/s]


Evaluating, analyzing and improving...
Done!


In [31]:
print(learn_results.experience.updated_instructions)

Classify a product review considering the following aspects: if the review consists of factual information or concrete feature descriptions, categorize it as "Objective"; if the review includes personal feelings, opinions, beliefs, or preferences of the reviewer, categorize it as "Subjective".

Examples:
Input: "The phone doesn't seem to accept anything except CBR mp3s."
Output: "Objective"
Input: "Not loud enough and doesn't turn on like it should."
Output: "Objective"
Input: "I don't like the color of this phone."
Output: "Subjective"
